In [14]:
import os
import time
import torch
import pickle as pkl
from importlib import import_module

In [2]:
dataset = 'data'  # 数据集
model_path = "./data/saved_dict/TextCNN.ckpt" # 模型
vocab_path = "./data/data/vocab.pkl"          # 词汇表 
embedding = 'embedding_Weibo.npz'             # 裁剪后的embedding
model_name = "TextCNN"                        # 模型名称
PAD = "<PAD>"
UNK = '<UNK>'

In [3]:
def load_vocab(vocab_path):
    """加载词表"""
    if os.path.exists(vocab_path):
        vocab = pkl.load(open(vocab_path, 'rb'))
        return vocab
    
def build_bacth(vocab, s, pad_size=32):
    """构建输入模型的数据"""
    token = [vocab.get(x, vocab.get(UNK)) for x in s]
    seq_len = len(token)
    if len(token) < pad_size:
        token.extend([vocab.get(PAD)] * (pad_size - len(token)))
    else:
        token = token[:pad_size]
        seq_len = len(token)
    token = torch.tensor([token]).to(config.device)
    seq_len = torch.tensor([[seq_len]])
    return (token, seq_len)

def predic(model, x):
    """预测"""
    with torch.no_grad():
        output = model(x)
        y = torch.max(output.data, 1)[1].cpu().numpy()
    return y

In [4]:
vocab = load_vocab(vocab_path)
module = import_module('models.' + model_name)
config = module.Config(dataset, embedding)

In [5]:
model = module.Model(config)

In [6]:
model.load_state_dict(torch.load(model_path))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [7]:
model = model.to(config.device)

In [8]:
model.eval()

Model(
  (embedding): Embedding(4075, 300)
  (convs): ModuleList(
    (0): Conv2d(1, 256, kernel_size=(2, 300), stride=(1, 1))
    (1): Conv2d(1, 256, kernel_size=(3, 300), stride=(1, 1))
    (2): Conv2d(1, 256, kernel_size=(4, 300), stride=(1, 1))
  )
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=768, out_features=2, bias=True)
)

In [20]:
s = "宝马这个车还可以啊，推荐买它"

In [23]:
start = time.time()
x = build_bacth(vocab, s, pad_size=config.pad_size)
y = predic(model, x)
end = time.time()
print(end-start)

0.0018589496612548828


In [24]:
y[-1]

1